<!-- Notebook Header Start -->

<h1 align="center">Spike Sorting</h1>

<p align="center">
  <strong>Author:</strong> Karl Bates<br>
  <strong>Date:</strong> 2024-10-30<br>
  <strong>Affiliation:</strong> Carnegie Mellon University, Cohen-Karni Group  || Neuromechatronics Lab
</p>

---

## 📜 Project Overview

(to populate) 

- **Objective:** (to populate)
- **Scope:** (to populate)
- **Libraries:** `Python`, `Pandas`, `SciPy`

---

## 📊 Notebook Outline

1. **Data Import & Preprocessing**


---

## 📚 References & Additional Resources

- [Github: Kilosort4](https://github.com/MouseLand/Kilosort/tree/main)
- [SpikeInterface](https://github.com/SpikeInterface)

---

<!-- Notebook Header End -->


# Libraries

In [6]:
# Imports
from pathlib import Path
from tqdm import tqdm
import numpy as np

# Download channel maps for default probes
from kilosort.utils import download_probes
download_probes()

from spikeinterface.extractors import read_intan

from kilosort import run_kilosort
from kilosort import io

# File Definitions

In [10]:
RAT_NAME = 'DW322'
STIMULATION = 'DRGS_2_240918_130024'

# Data Import

## Using `SpikeInterface`

In [7]:
# Specify the path where the data will be copied to, and where Kilosort4 results will be saved.
DATA_DIRECTORY = Path(r'D:\Data\CMU.80 Data\81 Data Inbox\DW322')  # NOTE: You should change this
# Create path if it doesn't exist
DATA_DIRECTORY.mkdir(parents=True, exist_ok=True)

In [ ]:

# NOTE: You will need to select the appropriate data stream. If you run without
#       specifying `stream_id`, you will get an error message explaining what
#       each stream corresponds to.
filepath = Path(f"D:\Data\DW322\DW322\DRGS_2_240918_130024\DRGS_2_240918_130024.rhd")
recording = read_intan(filepath, stream_id='0')

In [9]:
# NOTE: Data will be saved as np.int16 by default since that is the standard
#       for ephys data. If you need a different data type for whatever reason
#       such as `np.uint16`, be sure to update this.
dtype = np.int16
filename, N, c, s, fs, probe_path = io.spikeinterface_to_binary(
    recording, DATA_DIRECTORY, data_name='data.bin', dtype=dtype,
    chunksize=60000, export_probe=True, probe_name='probe.prb'
    )